<a href="https://colab.research.google.com/github/vladsavelyev/deeplearning/blob/master/pretgpt/pretgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
drive_path = Path("/content/drive/MyDrive")
!pip install torch transformers datasets tokenizers evaluate wandb peft

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from pathlib import Path
import numpy as np

import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments, TrainerCallback
from peft import get_peft_model, LoraConfig, TaskType

use_peft = True

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForCausalLM.from_pretrained(name)
print(f"Model parameters: {model.num_parameters():,}")
if use_peft:
    model = get_peft_model(model, LoraConfig(
        task_type=TaskType.CAUSAL_LM, 
        inference_mode=False, 
        r=8, 
        lora_alpha=32, 
        lora_dropout=0.1
    ))
    print("Parameter-efficient fine tuning trainable parameters:")
    model.print_trainable_parameters()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model parameters: 125,231,616
Parameter-efficient fine tuning trainable parameters:
trainable params: 294912 || all params: 125526528 || trainable%: 0.23493998017693918


In [4]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelForCausalLM.from_pretrained("gpt2")
model.config.vocab_size

50264

In [5]:
len(tokenizer.vocab)

50258

In [6]:
class TransformerDataset(Dataset):
    def __init__(self, token_ids: np.memmap, n_ctx: int):
        self.token_ids = token_ids
        self.n_ctx = n_ctx

    def __getitem__(self, idx):
        t = torch.LongTensor(self.token_ids[idx:idx + self.n_ctx])
        return {"input_ids": t, "labels": t}

    def __len__(self):
        return len(self.token_ids) - self.n_ctx + 1

    @staticmethod
    def load(path, tokenizer, n_ctx: int, max_n_examples: int = None) -> 'TransformerDataset':
        save_path = path.with_suffix(".token_ids.pt")
        if False and save_path.exists():
            print(f"Loading dataset from {save_path}")
            ids = torch.load(str(save_path))
        else:
            with open(path, "r") as f:
                text = f.read()
                print(f"Characters in text: {len(text):,}")
            ids = tokenizer(text, return_tensors="pt")['input_ids'].squeeze().long()
            if max_n_examples:
                max_tokens = max_n_examples + n_ctx - 1
                print(f"Taking first {max_tokens} tokens to make it {max_n_examples} examples")
                ids = ids[:max_tokens]
            eos = torch.tensor([tokenizer.eos_token_id]).long()
            ids = torch.concat((ids, eos))
            torch.save(ids, save_path)
        print(f"Dataset shape: {ids.shape}")
        return TransformerDataset(ids, n_ctx)


test_text_path = drive_path / "AI" / "datasets" / "murakami" / "murakami_test.txt"
train_text_path = drive_path / "AI" / "datasets" / "murakami" / "murakami_train.txt"
test_set = TransformerDataset.load(test_text_path, tokenizer, model.config.n_ctx, max_n_examples=100)
train_set = TransformerDataset.load(train_text_path, tokenizer, model.config.n_ctx)

Characters in text: 27,685
Taking first 2147 tokens to make it 100 examples
Dataset shape: torch.Size([2148])
Characters in text: 10,127,380
Dataset shape: torch.Size([2503610])


In [7]:
test_set.token_ids.min(), test_set.token_ids.max()

(tensor(5), tensor(50257))

In [8]:
train_set[0]['input_ids'].shape

torch.Size([2048])

In [9]:
print(f'{sum([p.numel() for p in model.parameters()]):,}')

125,526,528


In [10]:
model.config.n_ctx

2048

In [11]:
b = train_set[1000]
res = model(input_ids=b['input_ids'].view(1, -1), labels=b['labels'].view(1, -1))
print(res['loss'])

t = tokenizer("Определенно, это случилось с нами не в первый раз")['input_ids']
t = torch.LongTensor(t).view(1, -1)
res = model(input_ids=t, labels=t)
print(res['loss'])

tensor(3.0312, grad_fn=<NllLossBackward0>)
tensor(2.8863, grad_fn=<NllLossBackward0>)


In [12]:
from transformers.trainer_utils import get_last_checkpoint

save_dir = drive_path / ("AI/pretgpt/murakami_rugpt3small" + ("_peft" if use_peft else ""))
save_dir.mkdir(exist_ok=True)
if (last_checkpoint_dir := get_last_checkpoint(str(save_dir))):
    last_checkpoint_dir = Path(last_checkpoint_dir)
    print(last_checkpoint_dir)
    print('  '.join(t.name for t in last_checkpoint_dir.iterdir()))

In [13]:
# from transformers.utils import WEIGHTS_NAME
# state_dict = torch.load(str(last_checkpoint_dir / WEIGHTS_NAME), map_location="cpu")
# model.load_state_dict(state_dict)

In [21]:
def sample(num_seqs=5, max_length=20):
    torch.manual_seed(100)
    torch.cuda.manual_seed_all(100)
    for i, seq in enumerate(model.generate(
        max_length=max_length,
        ### -- my search --
        num_return_sequences=num_seqs,
        do_sample=True, 
        top_k=50,
        top_p=0.95,
        ### -- contrastive search? --
        # penalty_alpha=0.6, 
        # top_k=4,
        ### --
        pad_token_id=0,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )):
        print(i + 1, tokenizer.decode(seq))

sample(num_seqs=1, max_length=1000)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



1 <|endoftext|> с ее губ. Но ничего не сказала, лишь молча слушала, как она выкладывала на плоскую поверхность начатую книгу.
– Спасибо, – поблагодарил я. – А-а-а!
Долго не могли уснуть, а в душе все так же кипело и скреблось, как будто кто-то сказал: «Успокой, малыш. Вот и не думай больше о любви».
Проснулся от шума в коридоре. Зашел в комнату, увидел в окне «Пятьдесят шестой».
– И сколько раз я так стоял на вокзале? – спросил я.
К моему удивлению, ответ был один и тот же.
– И сколько раз ты так стоял? – повторил я. – И сколько раз так же стоял на вокзале?
– И все пять раз!
– Я тебя понимаю.
– Я бы сказал, что это примерно как: «Вот и не думай ни о чем… Просто подумай о том, чтобы больше никогда не иметь такого вопроса…»
– Да, вот и я не знаю.
– Просто вспомни.
И в самом деле, вспомнилось.
После двух дней в поезде я не переставал думать о ней. Она сидела у телефона с выключенным телефоном.
Я положил трубку. В трубке слышался ее голос.
– Как прошел сегодняшний день? – спросил я.
– Норм

In [15]:
import os
os.environ['WANDB_API_KEY'] = '11ef4840a7a0bfa6c0e28d71b539a6696d42bea5'
os.environ["WANDB_NOTEBOOK_NAME"] = "pretgpt"
from importlib import reload
import wandb
reload(wandb)

<module 'wandb' from '/usr/local/lib/python3.9/dist-packages/wandb/__init__.py'>

In [16]:
class MyCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        if metrics := kwargs.get("metrics"):
            print(f'Eval loss so far: {metrics["eval_loss"]:.4f}')
        if state.best_metric:
            print(f"Best loss so far: {state.best_metric:.4f}")
        sample()


trainer = Trainer(
    model=model,
    train_dataset=train_set,
    eval_dataset=test_set,
    callbacks=[MyCallback],
    args=TrainingArguments(
        output_dir=str(save_dir),
        report_to=['wandb'] if os.getenv("WANDB_API_KEY") else None,
        overwrite_output_dir=True,
        evaluation_strategy="steps",
        eval_steps=1000,
        save_steps=1000,
        save_total_limit=2,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        ignore_data_skip=True,
    ),
)
trainer.train(resume_from_checkpoint=last_checkpoint_dir)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2501563
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 3752346
  Number of trainable parameters = 294912
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find pretgpt.
wandb: Currently logged in as: vsavelyev. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
1000,3.152200,2.785661
2000,3.147200,2.787948
3000,3.131100,2.795480
4000,3.119300,2.796570


***** Running Evaluation *****
  Num examples = 101
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



Eval loss so far: 2.7857
1 <|endoftext|> у окна. Он подошел к краю окна и заглянул в комнату. Никого, кроме его, здесь
2 <|endoftext|> в этой жизни все-таки немного. На том берегу еще несколько домов. Я еще не добрался
3 <|endoftext|>.
— Наверно, это, — сказала она.
— А откуда
4 <|endoftext|> в сторону, и мне показалось, что я видел на горизонте ее силуэт.
— Ты…
5 <|endoftext|> из нее,
Я его как раз и нашел.
Такой же легкий, как и


Saving model checkpoint to /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small_peft/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 101
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



...

Eval loss so far: 2.7955
1 <|endoftext|>, и ее взгляд упал на меня. Однако я не стал тратить время, потому что, прежде
2 <|endoftext|> в комнату и закрыл за собой дверь. На следующее утро я, как всегда, проснулась от стука
3 <|endoftext|>.
— Если вы хотите продолжить, прошу, — сказала она.
— И, конечно
4 <|endoftext|> обратно в кровать и лечь, ожидая, пока все уснут. Но он не спал. И
5 <|endoftext|> себе, у меня отнялась рука, и рука больше не слушалась. Пришлось даже вызывать ее


Saving model checkpoint to /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small_peft/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [/content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small_peft/checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 101
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



Eval loss so far: 2.7966
1 <|endoftext|>, потом ее вернули на прежнее место. Однако в моем случае на нем отпечатков пальцев не оказалось
2 <|endoftext|> в комнату и закрыл за собой дверь. На следующее утро я, как всегда, проснулась от стука
3 <|endoftext|>.
— Если вы хотите продолжить, прошу, — сказала она.
— И, конечно
4 <|endoftext|> обратно в кровать и лечь, ожидая, пока все уснет. Но он не спал. И
5 <|endoftext|> из нее,
Я увидел, как она стала подниматься. – Ну-ка, еще раз


Saving model checkpoint to /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small_peft/checkpoint-4000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [/content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small_peft/checkpoint-2000] due to args.save_total_limit


KeyboardInterrupt: ignored

Parameter-efficient search, 4000 iterations:

In [17]:
sample(num_seqs=1, max_length=1000)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



1 <|endoftext|> не мог ничего разобрать, но все же старался быть предельно вежливым, старался помочь человеку.
Причем не в первый раз, с тех пор как мы встречались, ему все казалось каким-то безрадостным. В последнее время я заметил это, хотя внешне это не касалось. Что мы оба так себя ведем? Я и правда никогда в жизни не чувствовал такого – ни до, ни после. Ни капли в сравнении. Просто мне было до боли приятно чувствовать, что он все чувствует. Умеет ли он доверять мне? Понимает ли, что значит мой путь? Если и понимает, то насколько глубоко он в это погружен. Может, потому, что я тоже когда-то был в его глазах чужим?
Я снова начал звонить, чтобы задать его вопросы, но ответа все не последовало. Все казалось, что все просто в порядке, и он ждет ответа. Я так и не дождался. Может, он уже знает, что с ним произошло и что мне ответить? В любом случае, все же лучше для начала узнать что-то одно.
Я положил трубку и снова взялся за телефон. Все оказалось куда проще и проще. «Я же вам уже пис

Full model fine-tuning, 11000 iterations:

In [ ]:
sample(num_seqs=1, max_length=1000)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



1 <|endoftext|>-флейте у моей двери была дверь в ванную. В коридоре я стоял на шаг-парадном и разглядывал цветочки.
– Это, похоже, тюльпан, – сказал я.
– Бальзаматы? – Она обернулась и взглянула на меня так, словно видела перед собой какую-то редкую зверушку.
– Угу.
– Ты что, на тюленье окно похож?
– Конечно.
– А это что, действительно тюльпан?
– Конечно.
– А откуда он растет?
Я пошарил в карманах куртки, нащупал коробку с тюльпаном и тюльпаном и снова дернув за ручку, открыл дверь. Пока я искал тюльпан и вешалку, дождь все лил и лил – настолько обильно, что в ванной заискрился шербет. «Тюльпановы» у меня дома – коллекция еще с тех времен, когда мне семья выдраного корчилола глазки.
– 
В ванной мы оба долго ждали, пока он выйдет, – чтобы обнаружить, что кто-то снова там появится.
В конце концов дверь приоткрылась и стряхнула с себя капли. На миг в ванной посветлели, поблекли и померкли все краски. Сдохнув, дверь подалась на петлях, и на меня в окутавшую нас влагу вдруг пролился томител

### Comparing different of search strategies (for full model fine tuning)

In [ ]:
from pathlib import Path

from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

from transformers.trainer_utils import get_last_checkpoint
save_dir = DRIVE_PATH / "AI" / "pretgpt" / "murakami_rugpt3small"
save_dir.mkdir(exist_ok=True)
if last_checkpoint_dir := get_last_checkpoint(str(save_dir)):
    last_checkpoint_dir = Path(last_checkpoint_dir)
    print(last_checkpoint_dir)
    print('  '.join(t.name for t in last_checkpoint_dir.iterdir()))

from transformers.utils import WEIGHTS_NAME
import torch
state_dict = torch.load(str(last_checkpoint_dir / WEIGHTS_NAME), map_location="cpu")
model.load_state_dict(state_dict)    

def generate(**kwargs):
    torch.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    seq = model.generate(
        max_length=500,
        num_return_sequences=1,
        pad_token_id=0,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        **kwargs,
    )[0]
    print(tokenizer.decode(seq))
    print()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-11000
config.json  generation_config.json  pytorch_model.bin  training_args.bin  optimizer.pt  scheduler.pt  trainer_state.json  rng_state.pth


In [ ]:
print("My search:")
generate(
    do_sample=True, 
    top_p=0.95,
    top_k=50,
)

My search:
<|endoftext|>, а волосы были пострижены коротко и совсем коротко. А сам он, долговязый, как толстокожий, почти не подавал виду, но стоило ему приподнять голову, как на его лице появлялась какая-то удивительная улыбка. Всякий раз, когда я его замечал, в ответ эти губы складывались в неопределенную улыбку, которая мне так нравился.
– Ты здесь много времени проводишь? – поинтересовался я.
– А ты разве не в Токио?
– В Токио я был последний раз, – ответил я.
– И где же?
– Не знаю. Но не так далеко, к западу от Токио.
Я вспомнил, как мы расстались прошлым летом, как ходили вдвоем в поход по горам Яманотэ. Вспомнил и тот жаркий день – когда мы расстались с Кидзуки.
– А ты почему развелся? – спросил я.
В ответ – молчание. Казалось, она о чем-то задумалась.
– А я здесь пять лет жил. Работал на складе.
– Мой холодильник не работает.
– Извини. Я должен идти. А ты?
Я покачал головой.
– Знаешь… – сказала она. – Когда все успокоится, я уйду из этого дома. Останусь здесь, здесь, а не здесь

In [ ]:
print("Greedy search:")
generate()

Greedy search:
<|endoftext|>.
– А что, если я попробую с ним поговорить?
– Конечно, – ответил я. – Только учти, что я не смогу говорить с ним, пока ты не встретишься с ним.
– Хорошо, – сказала она. – Но учтите, что я не смогу поговорить с ним, пока ты не встретишься с ним.
– Хорошо, – сказал я.
Она положила руку мне на плечо.
– Я не знаю, как это сказать… но мне очень страшно.
– Не страшно?
– Да.
– Мне очень страшно, – повторила она. – Не представляю, что это значит.
– Я не знаю, что это значит, но… мне очень страшно.
– Ну, не знаю… – сказала она. – Но это значит, что я не смогу ему помочь.
– Каким образом?
– Не знаю. Но я уверена, что он – это он.
– И он – твой друг?
– Да.
– Значит, он – твой друг?
– Да.
– Значит, я смогу с ним поговорить?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Ду

In [ ]:
print("Nucleus sampling:")
generate(
    do_sample=True,
    top_p=0.05,
    top_k=0,
)

Nucleus sampling:
<|endoftext|>.
– А что, если я попробую с ним поговорить?
– Конечно, – ответил я. – Только учти, что я не смогу говорить с ним, пока ты не встретишься с ним.
– Хорошо, – сказала она. – Но учтите, что я не смогу поговорить с ним, пока ты не встретишься с ним.
– Хорошо, – сказал я.
Она положила руку мне на плечо.
– Я не знаю, как это сказать… но мне очень страшно.
– Не страшно?
– Да.
– Мне очень страшно, – повторила она. – Не представляю, что это значит.
– Я не знаю, что это значит, но… мне очень страшно.
– Ну, не знаю… – сказала она. – Но это значит, что я не смогу ему помочь.
– Каким образом?
– Не знаю. Но я уверена, что он – это он.
– И он – твой друг?
– Да.
– Значит, он – твой друг?
– Да.
– Значит, я смогу с ним поговорить?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
–

In [ ]:
print("Contrastive search:")
generate(
    penalty_alpha=0.6, 
    top_k=4,
)

Contrastive search:
<|endoftext|> в том, что это место обладает мощной энергией.
– Мощной?
– Да, излучающей волны высокой четкости. Она излучает не частоту, а импульс длительностью порядка десяти секунд. У меня в памяти это называется «параллельный мир». Но далеко не все, что я знаю, связано с этим местом. Есть у него какая-то связь со мной, а есть ли она у тебя, не знаю. Я думаю, это взаимосвязно.
– Что-то вроде родника? – предположил я.
– Нет, скорее всего, нет. Источник находится где-то на дне ущелья, окруженном высокими каменными стенами. И вода в нем – как из бочки, которую замуровали в склеп за кумирней. В мире, где отсутствуют звуки, которые можно слышать, и нет времени, чтобы насытиться этой водой. Она доступна только в подземном мире. Этот мир называется «Пространство А» и охраняется очень строгой дисциплиной. По религиозным соображениям туда не вход воспрещен. Однако верующие, католики и верующие-католики категорически против того, чтобы туда стекалась вода из других мест. Он

In [ ]:
print("Beam search:")
generate(
    do_sample=False,
    num_beams=4,
)

Beam search:
<|endoftext|>.
– Я не знаю, как это получилось, но, похоже, так оно и было, – сказал я.
– Я тоже так думаю, – сказала она.
– Я тоже, – сказал я.
– И все-таки?
– Все-таки, – сказал я.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней. – Все-таки, все-таки…
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки? – повторила она за мной.
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я 

In [ ]:
print("Beam multinomial search:")
generate(
    do_sample=True,
    num_beams=4,
)


Beam multinomial search:
<|endoftext|>.
– Ну, что? Поехали дальше?
– Да-да, конечно, – сказал я.
– Ну, что? Поехали дальше?
– Да-да, конечно, – сказал я.
– Ну, счастливо тебе, – сказала она.
– Пока, – ответил я.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – повторил я.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – повторил я.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – повторил я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – повторил я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказала она.
Пока, – сказала она.
Пока, – сказала она.
Пока, – сказала она.
Пока, – сказал я.
Пока, – сказала она.
Пока, – сказала она.
Пока, – сказала она.
По